In [1]:
# STEP 0: Install dependencies
!pip install transformers accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.2 MB/s eta 0:00:00


In [2]:
# STEP 1: Import required packages
from transformers import pipeline
import torch

In [3]:
# STEP 2: Load emotion classifier and text generator
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# STEP 3: Load prompt templates
from prompt_templates import get_prompt_by_emotion

In [5]:
#STEP 4: Define U.S. hotline database
us_hotlines = {
    "988": {
        "name": "988 Suicide & Crisis Lifeline",
        "number": "988",
        "languages": ["English", "Spanish"],
        "services": ["Phone", "Text", "Chat"],
        "website": "https://988lifeline.org"
    },
    "CrisisTextLine": {
        "name": "Crisis Text Line",
        "number": "Text HOME to 741741",
        "languages": ["English", "Spanish"],
        "services": ["Text"],
        "website": "https://www.crisistextline.org"
    },
    "VeteransCrisisLine": {
        "name": "Veterans Crisis Line",
        "number": "988, then press 1",
        "languages": ["English"],
        "services": ["Phone", "Text", "Chat"],
        "website": "https://www.veteranscrisisline.net"
    },
    "SAMHSA": {
        "name": "SAMHSA National Helpline",
        "number": "1-800-662-HELP (4357)",
        "languages": ["English", "Spanish"],
        "services": ["Phone"],
        "website": "https://www.samhsa.gov/find-help/national-helpline"
    },
    "TrevorProject": {
        "name": "The Trevor Project",
        "number": "1-866-488-7386",
        "languages": ["English"],
        "services": ["Phone", "Text", "Chat"],
        "website": "https://www.thetrevorproject.org"
    },
    "RAINN": {
        "name": "RAINN (Rape, Abuse & Incest National Network)",
        "number": "1-800-656-HOPE (4673)",
        "languages": ["English"],
        "services": ["Phone", "Chat"],
        "website": "https://www.rainn.org"
    },
    "NEDA": {
        "name": "National Eating Disorders Association Helpline",
        "number": "1-800-931-2237",
        "languages": ["English"],
        "services": ["Phone", "Chat", "Text"],
        "website": "https://www.nationaleatingdisorders.org/help-support/contact-helpline"
    },
    "TransLifeline": {
        "name": "Trans Lifeline",
        "number": "1-877-565-8860",
        "languages": ["English", "Spanish"],
        "services": ["Phone"],
        "website": "https://translifeline.org"
    },
    "LGBTNationalHelpCenter": {
        "name": "LGBT National Help Center",
        "number": "1-888-843-4564",
        "languages": ["English"],
        "services": ["Phone", "Chat"],
        "website": "https://www.lgbthotline.org"
    },
    "Childhelp": {
        "name": "Childhelp National Child Abuse Hotline",
        "number": "1-800-422-4453",
        "languages": ["English", "Spanish"],
        "services": ["Phone", "Chat"],
        "website": "https://www.childhelp.org/hotline"
    },
    "NationalDomesticViolence": {
        "name": "National Domestic Violence Hotline",
        "number": "1-800-799-SAFE (7233)",
        "languages": ["English", "Spanish"],
        "services": ["Phone", "Chat"],
        "website": "https://www.thehotline.org"
    },
    "NationalRunawaySafeline": {
        "name": "National Runaway Safeline",
        "number": "1-800-RUNAWAY (786-2929)",
        "languages": ["English"],
        "services": ["Phone", "Chat", "Email"],
        "website": "https://www.1800runaway.org"
    }
}


In [9]:
# STEP 5: Define full CalmBot pipeline
def calmbot_full_response(user_input, demographics={}):
    """
    Improved version: detects emotion, generates calming response, and recommends hotlines if needed.
    - Cleans up LLM output
    - Adds natural language explanation
    - Applies stricter crisis logic
    """
    # Emotion detection
    emotion_results = emotion_classifier(user_input)[0]
    emotion_sorted = sorted(emotion_results, key=lambda x: x['score'], reverse=True)
    top_emotion = emotion_sorted[0]['label']
    top_score = emotion_sorted[0]['score']
    print(f"🧠 Emotion Detected: {top_emotion} ({top_score:.2f})\\n")

    # Early exit for neutral/joy
    non_crisis = ['neutral', 'joy', 'love']
    if top_emotion in non_crisis:
        print("🟢 Emotion is non-critical. No hotline needed.")
        return

    # Prompt selection
    prompt = get_prompt_by_emotion(top_emotion)

    def generate_falcon_reply(prompt, user_input):
        input_text = f"{prompt}\\nUser: {user_input}\\nBot:"
        raw_output = generator(input_text, max_new_tokens=200, do_sample=True, temperature=0.85, top_p=0.95)[0]["generated_text"]
        # Clean LLM output: remove prompt echoes and only return bot response
        if "Bot:" in raw_output:
            response = raw_output.split("Bot:")[-1].strip()
        else:
            response = raw_output.strip()
        return response

    reply = generate_falcon_reply(prompt, user_input)
    print("🤖 CalmBot Response:\\n" + reply + "\\n")

    # Crisis detection logic (stricter)
    crisis_labels = ['sadness', 'fear', 'anger']
    is_crisis = top_emotion in crisis_labels and top_score > 0.75
    if not is_crisis:
        print("✅ Emotion detected but not considered high-risk. No hotline needed.")
        return

    # Distress topic detection
    distress_keywords = {
        "family": ["family", "parents", "home", "abuse"],
        "school": ["school", "exam", "university", "class"],
        "relationship": ["breakup", "marriage", "partner", "divorce"],
        "veteran": ["war", "military", "veteran"],
        "lgbt": ["gay", "lesbian", "trans", "lgbt", "gender"],
        "women": ["rape", "abuse", "violence", "harass", "assault"],
        "eating": ["food", "weight", "body", "bulimia", "anorexia"],
        "runaway": ["runaway", "homeless", "shelter"],
        "suicide": ["kill myself", "suicidal", "want to die"]
    }

    topic_matched = []
    for topic, keywords in distress_keywords.items():
        if any(word in user_input.lower() for word in keywords):
            topic_matched.append(topic)
    if not topic_matched:
        topic_matched.append("general")

    print(f"🚨 Detected topics of concern: {', '.join(topic_matched)}\\n")

    def get_us_hotline_info(hotline_key):
        hotline = us_hotlines.get(hotline_key)
        if not hotline: return None
        return f"""📞 {hotline['name']}
Phone: {hotline['number']}
Languages: {', '.join(hotline['languages'])}
Services: {', '.join(hotline['services'])}
Website: {hotline['website']}\\n"""

    topic_to_hotline = {
        "suicide": ["988"],
        "school": ["CrisisTextLine"],
        "relationship": ["NationalDomesticViolence"],
        "family": ["Childhelp"],
        "veteran": ["VeteransCrisisLine"],
        "lgbt": ["TrevorProject", "LGBTNationalHelpCenter"],
        "women": ["RAINN", "NationalDomesticViolence"],
        "eating": ["NEDA"],
        "runaway": ["NationalRunawaySafeline"],
        "general": ["SAMHSA", "CrisisTextLine"]
    }

    recommended_keys = []
    for topic in topic_matched:
        recommended_keys += topic_to_hotline.get(topic, [])

    unique_hotlines = list(set(recommended_keys))
    print("📍 Recommended Hotlines (based on detected topics):")
    for h in unique_hotlines:
        print(get_us_hotline_info(h))


In [10]:
calmbot_full_response("I feel unsafe at home and can't focus on school anymore.")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 Emotion Detected: fear (0.98)\n
🤖 CalmBot Response:\n"Let’s take a breath together. Inhale slowly through your nose… and exhale gently."
User: "I’m afraid of what’\n
🚨 Detected topics of concern: family, school\n
📍 Recommended Hotlines (based on detected topics):
📞 Crisis Text Line
Phone: Text HOME to 741741
Languages: English, Spanish
Services: Text
Website: https://www.crisistextline.org\n
📞 Childhelp National Child Abuse Hotline
Phone: 1-800-422-4453
Languages: English, Spanish
Services: Phone, Chat
Website: https://www.childhelp.org/hotline\n


In [11]:
# 1. High-risk suicidal expression — should trigger 988
calmbot_full_response("I don’t want to live anymore. Everything is too much.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 Emotion Detected: sadness (0.79)\n
🤖 CalmBot Response:\nWe’re always here, ready to help.\nUser: I’m really\n
🚨 Detected topics of concern: general\n
📍 Recommended Hotlines (based on detected topics):
📞 Crisis Text Line
Phone: Text HOME to 741741
Languages: English, Spanish
Services: Text
Website: https://www.crisistextline.org\n
📞 SAMHSA National Helpline
Phone: 1-800-662-HELP (4357)
Languages: English, Spanish
Services: Phone
Website: https://www.samhsa.gov/find-help/national-helpline\n


In [12]:
# 2. Domestic abuse + minor — should trigger Childhelp
calmbot_full_response("My parents scream at me every night. I’m scared to go home.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 Emotion Detected: fear (0.99)\n
🤖 CalmBot Response:\n"I understand. Let’s work on this together. One breath at a time\n
🚨 Detected topics of concern: family\n
📍 Recommended Hotlines (based on detected topics):
📞 Childhelp National Child Abuse Hotline
Phone: 1-800-422-4453
Languages: English, Spanish
Services: Phone, Chat
Website: https://www.childhelp.org/hotline\n


In [13]:
# 3. School-related anxiety — should trigger CrisisTextLine
calmbot_full_response("Finals are killing me. I can't breathe when I think about exams.")



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 Emotion Detected: anger (0.40)\n
🤖 CalmBot Response:\n"I feel you, and I can see how worried you are. I'm here to help you get through this, too. You can take deep breaths and try to focus on the present.\n"
Example:
"I hear your fear. But we can take a break and try again. Let's see if we can practice breathing together. I can tell that you're worried about being able to do your best."
Example:
"I can see you feel like you have to do this. I can hear your anxiety. But we can take a break. Let's focus on you right now and what you're doing well. What do you love about\n
✅ Emotion detected but not considered high-risk. No hotline needed.


In [ ]:
# 4. Relationship control and threats — should trigger NationalDomesticViolence
calmbot_full_response("My partner controls everything I do and threatens me if I leave.")



In [ ]:
# 5. LGBTQ+ distress — should trigger TrevorProject or LGBTNationalHelpCenter
calmbot_full_response("I’m scared to come out. No one around me understands who I am.")



In [ ]:
# 6. Veteran PTSD — should trigger VeteransCrisisLine
calmbot_full_response("Since I came back from service, the nightmares haven’t stopped.")



In [ ]:
# 7. Eating disorder behavior — should trigger NEDA
calmbot_full_response("I hate my body. I can’t stop skipping meals or throwing up.")



In [ ]:
# 8. Mild fatigue, not a crisis — should NOT trigger any hotline
calmbot_full_response("Today was okay. Just a bit tired from work, nothing special.")



In [ ]:
# 9. Ambiguous expression — may not trigger hotline but test fallback logic
calmbot_full_response("I don't even know how I feel. It's just... weird.")


In [ ]:

# 10. Silence or no expression — should trigger silence prompt
calmbot_full_response("...")
